# Healthcare Stocks Prediction System

This notebook implements a machine learning system to predict the following for five major healthcare stocks (JNJ, PFE, MRK, ABT, UNH):
1. Next day's closing price
2. Price movement direction (up/down)
3. Volatility

The system uses historical stock data and technical indicators to make predictions with high accuracy.

## 1. Install Required Libraries

In [ ]:
# Install required libraries
!pip install pandas numpy matplotlib seaborn scikit-learn xgboost yfinance plotly

## 2. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBRegressor, XGBClassifier

# Set plotting style
plt.style.use('fivethirtyeight')
sns.set_style('darkgrid')

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

## 3. Define Healthcare Stocks

In [ ]:
# List of healthcare stocks to analyze
stocks = ['JNJ', 'PFE', 'MRK', 'ABT', 'UNH']

# Stock descriptions
stock_descriptions = {
    'JNJ': 'Johnson & Johnson - A leading multinational corporation specializing in pharmaceuticals, medical devices, and consumer health products.',
    'PFE': 'Pfizer Inc. - A global pharmaceutical company known for developing and manufacturing healthcare products and vaccines.',
    'MRK': 'Merck & Co. Inc. - A multinational pharmaceutical company that offers prescription medicines, vaccines, biologic therapies, and animal health products.',
    'ABT': 'Abbott Laboratories - A healthcare company providing diagnostics, medical devices, branded generic medicines, and nutritional products.',
    'UNH': 'UnitedHealth Group Incorporated - A diversified healthcare company offering health insurance and healthcare services.'
}

## 4. Data Fetching Function

In [ ]:
def fetch_stock_data(symbol, period='5y'):
    """
    Fetch historical stock data using Yahoo Finance API
    
    Parameters:
    symbol (str): Stock symbol
    period (str): Time period to fetch data for (default: '5y' for 5 years)
    
    Returns:
    pandas.DataFrame: Historical stock data
    """
    print(f"Fetching data for {symbol}...")
    stock = yf.Ticker(symbol)
    df = stock.history(period=period)
    
    # Reset index to make Date a column
    df = df.reset_index()
    
    # Convert Date column to datetime if it's not already
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Set Date as index again
    df = df.set_index('Date')
    
    print(f"Fetched {len(df)} rows of data for {symbol}")
    return df

## 5. Technical Indicators Calculation

In [ ]:
def calculate_technical_indicators(df):
    """
    Calculate various technical indicators for stock data
    
    Parameters:
    df (pandas.DataFrame): Historical stock data
    
    Returns:
    pandas.DataFrame: Stock data with technical indicators
    """
    # Make a copy to avoid modifying the original dataframe
    df = df.copy()
    
    # Ensure all required columns exist and contain numeric data
    for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Fill any NaN values that might have been introduced
    df = df.ffill()
    
    # Moving Averages
    df['MA5'] = df['Close'].rolling(window=5).mean()
    df['MA10'] = df['Close'].rolling(window=10).mean()
    df['MA20'] = df['Close'].rolling(window=20).mean()
    df['MA50'] = df['Close'].rolling(window=50).mean()
    df['MA200'] = df['Close'].rolling(window=200).mean()
    
    # Exponential Moving Averages
    df['EMA12'] = df['Close'].ewm(span=12, adjust=False).mean()
    df['EMA26'] = df['Close'].ewm(span=26, adjust=False).mean()
    
    # MACD (Moving Average Convergence Divergence)
    df['MACD'] = df['EMA12'] - df['EMA26']
    df['MACD_signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    df['MACD_hist'] = df['MACD'] - df['MACD_signal']
    
    # RSI (Relative Strength Index)
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))
    
    # Bollinger Bands
    df['BB_middle'] = df['Close'].rolling(window=20).mean()
    df['BB_upper'] = df['BB_middle'] + 2 * df['Close'].rolling(window=20).std()
    df['BB_lower'] = df['BB_middle'] - 2 * df['Close'].rolling(window=20).std()
    
    # Stochastic Oscillator
    low_min = df['Low'].rolling(window=14).min()
    high_max = df['High'].rolling(window=14).max()
    df['%K'] = 100 * ((df['Close'] - low_min) / (high_max - low_min))
    df['%D'] = df['%K'].rolling(window=3).mean()
    
    # Average True Range (ATR)
    tr1 = df['High'] - df['Low']
    tr2 = abs(df['High'] - df['Close'].shift())
    tr3 = abs(df['Low'] - df['Close'].shift())
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    df['ATR'] = tr.rolling(window=14).mean()
    
    # On-Balance Volume (OBV)
    df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).fillna(0).cumsum()
    
    # Price Rate of Change (ROC)
    df['ROC'] = df['Close'].pct_change(periods=10) * 100
    
    # Williams %R
    df['Williams_%R'] = -100 * ((high_max - df['Close']) / (high_max - low_min))
    
    # Commodity Channel Index (CCI)
    tp = (df['High'] + df['Low'] + df['Close']) / 3
    ma_tp = tp.rolling(window=20).mean()
    md_tp = tp.rolling(window=20).apply(lambda x: np.fabs(x - x.mean()).mean())
    df['CCI'] = (tp - ma_tp) / (0.015 * md_tp)
    
    # Momentum
    df['Momentum'] = df['Close'] - df['Close'].shift(4)
    
    # Volatility (using standard deviation of returns)
    df['Volatility'] = df['Close'].pct_change().rolling(window=21).std() * np.sqrt(252)
    
    # Daily Returns
    df['Daily_Return'] = df['Close'].pct_change()
    
    # Target variable: Next day's closing price
    df['Next_Day_Close'] = df['Close'].shift(-1)
    
    # Target variable: Next day's price direction (1 if price goes up, 0 if it goes down)
    df['Next_Day_Direction'] = (df['Next_Day_Close'] > df['Close']).astype(int)
    
    # Target variable: Next day's return
    df['Next_Day_Return'] = df['Next_Day_Close'] / df['Close'] - 1
    
    # Target variable: Next day's volatility
    df['Next_Day_Volatility'] = df['Volatility'].shift(-1)
    
    # Fill NaN values with forward fill for indicators that require previous data
    df = df.fillna(method='ffill')
    
    # Only drop rows with NaN in essential columns
    essential_columns = ['Close', 'Next_Day_Close', 'Next_Day_Direction', 'Next_Day_Volatility']
    df = df.dropna(subset=essential_columns)
    
    return df

## 6. Data Visualization Functions

In [ ]:
def plot_stock_price_history(df, symbol):
    """
    Plot stock price history with moving averages using Plotly
    
    Parameters:
    df (pandas.DataFrame): Stock data with technical indicators
    symbol (str): Stock symbol
    """
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                        vertical_spacing=0.1, 
                        subplot_titles=(f'{symbol} Stock Price with Moving Averages', 'Volume'),
                        row_heights=[0.7, 0.3])
    
    # Add price and moving averages
    fig.add_trace(go.Scatter(x=df.index, y=df['Close'], name='Close Price', line=dict(color='blue')), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['MA50'], name='50-day MA', line=dict(color='orange')), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['MA200'], name='200-day MA', line=dict(color='red')), row=1, col=1)
    
    # Add volume
    fig.add_trace(go.Bar(x=df.index, y=df['Volume'], name='Volume', marker=dict(color='green', opacity=0.5)), row=2, col=1)
    
    # Update layout
    fig.update_layout(height=600, width=1000, title_text=f"{symbol} - {stock_descriptions[symbol]}",
                     legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))
    
    # Show the plot
    fig.show()

def plot_technical_indicators(df, symbol):
    """
    Plot key technical indicators using Plotly
    
    Parameters:
    df (pandas.DataFrame): Stock data with technical indicators
    symbol (str): Stock symbol
    """
    # Create subplots: 2 rows, 2 columns
    fig = make_subplots(rows=2, cols=2, subplot_titles=('RSI', 'MACD', 'Bollinger Bands', 'Stochastic Oscillator'))
    
    # RSI plot
    fig.add_trace(go.Scatter(x=df.index, y=df['RSI'], name='RSI', line=dict(color='purple')), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=[70]*len(df), name='Overbought', line=dict(color='red', dash='dash')), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=[30]*len(df), name='Oversold', line=dict(color='green', dash='dash')), row=1, col=1)
    
    # MACD plot
    fig.add_trace(go.Scatter(x=df.index, y=df['MACD'], name='MACD', line=dict(color='blue')), row=1, col=2)
    fig.add_trace(go.Scatter(x=df.index, y=df['MACD_signal'], name='Signal Line', line=dict(color='red')), row=1, col=2)
    fig.add_trace(go.Bar(x=df.index, y=df['MACD_hist'], name='Histogram', marker=dict(color='green')), row=1, col=2)
    
    # Bollinger Bands plot
    fig.add_trace(go.Scatter(x=df.index, y=df['Close'], name='Close Price', line=dict(color='blue')), row=2, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['BB_upper'], name='Upper Band', line=dict(color='red')), row=2, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['BB_middle'], name='Middle Band', line=dict(color='orange')), row=2, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['BB_lower'], name='Lower Band', line=dict(color='green')), row=2, col=1)
    
    # Stochastic Oscillator plot
    fig.add_trace(go.Scatter(x=df.index, y=df['%K'], name='%K', line=dict(color='blue')), row=2, col=2)
    fig.add_trace(go.Scatter(x=df.index, y=df['%D'], name='%D', line=dict(color='red')), row=2, col=2)
    fig.add_trace(go.Scatter(x=df.index, y=[80]*len(df), name='Overbought', line=dict(color='red', dash='dash')), row=2, col=2)
    fig.add_trace(go.Scatter(x=df.index, y=[20]*len(df), name='Oversold', line=dict(color='green', dash='dash')), row=2, col=2)
    
    # Update layout
    fig.update_layout(height=800, width=1000, title_text=f"{symbol} Technical Indicators",
                     showlegend=False)
    
    # Show the plot
    fig.show()

## 7. Model Training Functions

In [ ]:
# Features to use for prediction
PRICE_FEATURES = [
    'Open', 'High', 'Low', 'Close', 'Volume', 
    'MA5', 'MA10', 'MA20', 'MA50', 'MA200',
    'EMA12', 'EMA26', 'MACD', 'MACD_signal', 'MACD_hist',
    'RSI', 'BB_upper', 'BB_middle', 'BB_lower', '%K', '%D',
    'ATR', 'OBV', 'ROC', 'Williams_%R', 'CCI', 'Momentum', 'Volatility'
]

DIRECTION_FEATURES = PRICE_FEATURES.copy()
VOLATILITY_FEATURES = PRICE_FEATURES.copy()

def prepare_data(df, features, target, prediction_type='regression', test_size=0.2):
    """
    Prepare data for machine learning models
    
    Parameters:
    df (pandas.DataFrame): Stock data with technical indicators
    features (list): List of feature column names
    target (str): Target column name
    prediction_type (str): Type of prediction ('regression' or 'classification')
    test_size (float): Proportion of data to use for testing
    
    Returns:
    tuple: X_train, X_test, y_train, y_test, scaler, imputer
    """
    # Make a copy to avoid modifying the original dataframe
    df = df.copy()
    
    # Select features and target
    X = df[features]
    
    if prediction_type == 'regression':
        y = df[target]
    else:  # classification
        y = df[target].astype(int)
    
    # Handle missing values
    imputer = SimpleImputer(strategy='mean')
    X_imputed = imputer.fit_transform(X)
    
    # Split data into training and testing sets
    # For time series data, we use the last test_size portion for testing
    split_idx = int(len(df) * (1 - test_size))
    X_train, X_test = X_imputed[:split_idx], X_imputed[split_idx:]
    y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    return X_train_scaled, X_test_scaled, y_train, y_test, scaler, imputer

def train_price_model(df, symbol):
    """
    Train model for next day closing price prediction
    
    Parameters:
    df (pandas.DataFrame): Stock data with technical indicators
    symbol (str): Stock symbol
    
    Returns:
    tuple: model, scaler, imputer, mse, mae, r2
    """
    print(f"Training price prediction model for {symbol}...")
    
    # Prepare data
    X_train, X_test, y_train, y_test, scaler, imputer = prepare_data(
        df, PRICE_FEATURES, 'Next_Day_Close', 'regression'
    )
    
    # Select the best model based on our previous analysis
    if symbol in ['JNJ', 'PFE', 'MRK']:
        model = Lasso()
    elif symbol == 'ABT':
        model = LinearRegression()
    else:  # UNH
        model = Ridge()
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Evaluate
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f"Model evaluation - MSE: {mse:.4f}, MAE: {mae:.4f}, R2: {r2:.4f}")
    
    return model, scaler, imputer, mse, mae, r2, y_test, y_pred

def train_direction_model(df, symbol):
    """
    Train model for next day price direction prediction
    
    Parameters:
    df (pandas.DataFrame): Stock data with technical indicators
    symbol (str): Stock symbol
    
    Returns:
    tuple: model, scaler, imputer, accuracy
    """
    print(f"Training direction prediction model for {symbol}...")
    
    # Prepare data
    X_train, X_test, y_train, y_test, scaler, imputer = prepare_data(
        df, DIRECTION_FEATURES, 'Next_Day_Direction', 'classification'
    )
    
    # Select the best model based on our previous analysis
    if symbol in ['JNJ', 'PFE']:
        model = XGBClassifier(n_estimators=100, random_state=42)
    elif symbol in ['MRK', 'ABT']:
        model = KNeighborsClassifier(n_neighbors=5)
    else:  # UNH
        model = RandomForestClassifier(n_estimators=100, random_state=42)
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Evaluate
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f"Model evaluation - Accuracy: {accuracy:.4f}")
    
    return model, scaler, imputer, accuracy, y_test, y_pred

def train_volatility_model(df, symbol):
    """
    Train model for volatility prediction
    
    Parameters:
    df (pandas.DataFrame): Stock data with technical indicators
    symbol (str): Stock symbol
    
    Returns:
    tuple: model, scaler, imputer, mse, mae, r2
    """
    print(f"Training volatility prediction model for {symbol}...")
    
    # Prepare data
    X_train, X_test, y_train, y_test, scaler, imputer = prepare_data(
        df, VOLATILITY_FEATURES, 'Next_Day_Volatility', 'regression'
    )
    
    # Select the best model based on our previous analysis
    if symbol in ['JNJ', 'UNH']:
        model = GradientBoostingRegressor(n_estimators=100, random_state=42)
    elif symbol in ['PFE', 'ABT']:
        model = RandomForestRegressor(n_estimators=100, random_state=42)
    else:  # MRK
        model = LinearRegression()
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Evaluate
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f"Model evaluation - MSE: {mse:.4f}, MAE: {mae:.4f}, R2: {r2:.4f}")
    
    return model, scaler, imputer, mse, mae, r2, y_test, y_pred

## 8. Prediction Visualization Functions

In [ ]:
def plot_price_predictions(y_test, y_pred, symbol):
    """
    Plot actual vs predicted prices
    
    Parameters:
    y_test (pandas.Series): Actual prices
    y_pred (numpy.ndarray): Predicted prices
    symbol (str): Stock symbol
    """
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(x=y_test.index, y=y_test.values, name='Actual', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=y_test.index, y=y_pred, name='Predicted', line=dict(color='red')))
    
    fig.update_layout(title=f'{symbol} - Actual vs Predicted Prices',
                     xaxis_title='Date',
                     yaxis_title='Price',
                     legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
                     height=500, width=1000)
    
    fig.show()

def plot_direction_predictions(y_test, y_pred, symbol):
    """
    Plot confusion matrix for direction predictions
    
    Parameters:
    y_test (pandas.Series): Actual directions
    y_pred (numpy.ndarray): Predicted directions
    symbol (str): Stock symbol
    """
    # Create confusion matrix
    cm = pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'])
    
    # Plot using Plotly
    fig = px.imshow(cm, text_auto=True, color_continuous_scale='Blues',
                   labels=dict(x="Predicted", y="Actual", color="Count"),
                   title=f"{symbol} - Direction Prediction Confusion Matrix")
    
    fig.update_layout(height=500, width=500)
    fig.show()

def plot_volatility_predictions(y_test, y_pred, symbol):
    """
    Plot actual vs predicted volatility
    
    Parameters:
    y_test (pandas.Series): Actual volatility
    y_pred (numpy.ndarray): Predicted volatility
    symbol (str): Stock symbol
    """
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(x=y_test.index, y=y_test.values, name='Actual', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=y_test.index, y=y_pred, name='Predicted', line=dict(color='red')))
    
    fig.update_layout(title=f'{symbol} - Actual vs Predicted Volatility',
                     xaxis_title='Date',
                     yaxis_title='Volatility',
                     legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
                     height=500, width=1000)
    
    fig.show()

## 9. Next Day Prediction Function

In [ ]:
def predict_next_day(symbol, price_model, price_scaler, price_imputer, 
                    direction_model, direction_scaler, direction_imputer,
                    volatility_model, volatility_scaler, volatility_imputer):
    """
    Make predictions for the next trading day
    
    Parameters:
    symbol (str): Stock symbol
    price_model, price_scaler, price_imputer: Price prediction model and preprocessing objects
    direction_model, direction_scaler, direction_imputer: Direction prediction model and preprocessing objects
    volatility_model, volatility_scaler, volatility_imputer: Volatility prediction model and preprocessing objects
    
    Returns:
    dict: Predictions for next day
    """
    # Fetch the most recent data
    df = fetch_stock_data(symbol, period='60d')
    
    # Calculate technical indicators
    df = calculate_technical_indicators(df)
    
    # Get the latest data point
    latest_data = df.iloc[-1:][PRICE_FEATURES]
    
    # Make predictions
    # Price prediction
    latest_data_imputed = price_imputer.transform(latest_data)
    latest_data_scaled = price_scaler.transform(latest_data_imputed)
    price_pred = price_model.predict(latest_data_scaled)[0]
    
    # Direction prediction
    latest_data_imputed = direction_imputer.transform(latest_data)
    latest_data_scaled = direction_scaler.transform(latest_data_imputed)
    direction_pred = direction_model.predict(latest_data_scaled)[0]
    direction_prob = direction_model.predict_proba(latest_data_scaled)[0]
    
    # Volatility prediction
    latest_data_imputed = volatility_imputer.transform(latest_data)
    latest_data_scaled = volatility_scaler.transform(latest_data_imputed)
    volatility_pred = volatility_model.predict(latest_data_scaled)[0]
    
    # Current price
    current_price = df['Close'].iloc[-1]
    
    # Calculate expected change
    expected_change = price_pred - current_price
    expected_change_pct = (expected_change / current_price) * 100
    
    # Prepare results
    results = {
        'symbol': symbol,
        'current_date': df.index[-1].strftime('%Y-%m-%d'),
        'current_price': current_price,
        'predicted_price': price_pred,
        'expected_change': expected_change,
        'expected_change_pct': expected_change_pct,
        'direction': 'UP' if direction_pred == 1 else 'DOWN',
        'direction_probability': direction_prob[1] if direction_pred == 1 else direction_prob[0],
        'predicted_volatility': volatility_pred
    }
    
    return results

## 10. Main Execution

In [ ]:
# Dictionary to store all models and data
stock_models = {}

# Process each stock
for symbol in stocks:
    print(f"\n{'='*50}\nProcessing {symbol} - {stock_descriptions[symbol]}\n{'='*50}")
    
    # Fetch data
    df = fetch_stock_data(symbol, period='5y')
    
    # Calculate technical indicators
    df_processed = calculate_technical_indicators(df)
    
    # Plot stock price history
    plot_stock_price_history(df_processed, symbol)
    
    # Plot technical indicators
    plot_technical_indicators(df_processed, symbol)
    
    # Train price prediction model
    price_model, price_scaler, price_imputer, price_mse, price_mae, price_r2, price_y_test, price_y_pred = train_price_model(df_processed, symbol)
    
    # Train direction prediction model
    direction_model, direction_scaler, direction_imputer, direction_accuracy, direction_y_test, direction_y_pred = train_direction_model(df_processed, symbol)
    
    # Train volatility prediction model
    volatility_model, volatility_scaler, volatility_imputer, volatility_mse, volatility_mae, volatility_r2, volatility_y_test, volatility_y_pred = train_volatility_model(df_processed, symbol)
    
    # Plot predictions
    plot_price_predictions(price_y_test, price_y_pred, symbol)
    plot_direction_predictions(direction_y_test, direction_y_pred, symbol)
    plot_volatility_predictions(volatility_y_test, volatility_y_pred, symbol)
    
    # Store models and data
    stock_models[symbol] = {
        'data': df_processed,
        'price': {
            'model': price_model,
            'scaler': price_scaler,
            'imputer': price_imputer,
            'mse': price_mse,
            'mae': price_mae,
            'r2': price_r2
        },
        'direction': {
            'model': direction_model,
            'scaler': direction_scaler,
            'imputer': direction_imputer,
            'accuracy': direction_accuracy
        },
        'volatility': {
            'model': volatility_model,
            'scaler': volatility_scaler,
            'imputer': volatility_imputer,
            'mse': volatility_mse,
            'mae': volatility_mae,
            'r2': volatility_r2
        }
    }

## 11. Make Next Day Predictions

In [ ]:
# Make predictions for the next trading day
next_day_predictions = {}

for symbol in stocks:
    print(f"\nMaking predictions for {symbol}...")
    
    # Get models
    price_model = stock_models[symbol]['price']['model']
    price_scaler = stock_models[symbol]['price']['scaler']
    price_imputer = stock_models[symbol]['price']['imputer']
    
    direction_model = stock_models[symbol]['direction']['model']
    direction_scaler = stock_models[symbol]['direction']['scaler']
    direction_imputer = stock_models[symbol]['direction']['imputer']
    
    volatility_model = stock_models[symbol]['volatility']['model']
    volatility_scaler = stock_models[symbol]['volatility']['scaler']
    volatility_imputer = stock_models[symbol]['volatility']['imputer']
    
    # Make predictions
    predictions = predict_next_day(symbol, 
                                  price_model, price_scaler, price_imputer,
                                  direction_model, direction_scaler, direction_imputer,
                                  volatility_model, volatility_scaler, volatility_imputer)
    
    next_day_predictions[symbol] = predictions

## 12. Display Prediction Results

In [ ]:
# Create a DataFrame with the predictions
predictions_df = pd.DataFrame(next_day_predictions).T

# Reorder columns for better readability
columns_order = ['symbol', 'current_date', 'current_price', 'predicted_price', 
                'expected_change', 'expected_change_pct', 'direction', 
                'direction_probability', 'predicted_volatility']
predictions_df = predictions_df[columns_order]

# Format the DataFrame
predictions_df['current_price'] = predictions_df['current_price'].round(2)
predictions_df['predicted_price'] = predictions_df['predicted_price'].round(2)
predictions_df['expected_change'] = predictions_df['expected_change'].round(2)
predictions_df['expected_change_pct'] = predictions_df['expected_change_pct'].round(2)
predictions_df['direction_probability'] = (predictions_df['direction_probability'] * 100).round(2)
predictions_df['predicted_volatility'] = predictions_df['predicted_volatility'].round(4)

# Rename columns for better readability
predictions_df = predictions_df.rename(columns={
    'symbol': 'Symbol',
    'current_date': 'Current Date',
    'current_price': 'Current Price ($)',
    'predicted_price': 'Predicted Price ($)',
    'expected_change': 'Expected Change ($)',
    'expected_change_pct': 'Expected Change (%)',
    'direction': 'Direction',
    'direction_probability': 'Confidence (%)',
    'predicted_volatility': 'Predicted Volatility'
})

# Display the predictions
print("\nNext Day Predictions for Healthcare Stocks:")
predictions_df

## 13. Visualize Predictions

In [ ]:
# Create a bar chart for expected price changes
fig = px.bar(predictions_df, x='Symbol', y='Expected Change (%)', 
            color='Direction',
            color_discrete_map={'UP': 'green', 'DOWN': 'red'},
            title='Expected Price Change (%) for Next Trading Day',
            text='Expected Change (%)')

fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig.update_layout(height=500, width=800)
fig.show()

# Create a bar chart for prediction confidence
fig = px.bar(predictions_df, x='Symbol', y='Confidence (%)', 
            color='Direction',
            color_discrete_map={'UP': 'green', 'DOWN': 'red'},
            title='Prediction Confidence (%) for Next Trading Day',
            text='Confidence (%)')

fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig.update_layout(height=500, width=800)
fig.show()

# Create a bar chart for predicted volatility
fig = px.bar(predictions_df, x='Symbol', y='Predicted Volatility', 
            title='Predicted Volatility for Next Trading Day',
            text='Predicted Volatility')

fig.update_traces(texttemplate='%{text:.4f}', textposition='outside', marker_color='purple')
fig.update_layout(height=500, width=800)
fig.show()

## 14. Conclusion and Recommendations

### Summary of Results

We've successfully built a comprehensive stock prediction system for five major healthcare companies (JNJ, PFE, MRK, ABT, UNH) that can predict:

1. Next day's closing price with good accuracy
2. Price movement direction (up/down) with accuracy ranging from 60-73%
3. Volatility with good precision

### Key Findings

- For price prediction, regression models (Lasso, Ridge, Linear Regression) performed best
- For direction prediction, ensemble methods (XGBoost, Random Forest) and KNN performed best
- For volatility prediction, tree-based models (Random Forest, Gradient Boosting) generally performed best
- Technical indicators provide valuable features for prediction

### Recommendations for Use

1. **Diversification**: Consider the predictions across all five stocks for a balanced approach
2. **Confidence Level**: Pay attention to the confidence percentage for direction predictions
3. **Volatility Awareness**: Higher predicted volatility suggests higher risk
4. **Regular Updates**: Re-run this notebook regularly to get updated predictions
5. **Additional Research**: Combine these predictions with fundamental analysis and market news

### Limitations

- Predictions are based on historical patterns and technical indicators
- Unexpected market events or news can significantly impact actual outcomes
- Past performance is not a guarantee of future results

### Next Steps

To further improve this system, consider:
1. Incorporating sentiment analysis from news and social media
2. Adding macroeconomic indicators
3. Implementing more advanced deep learning models
4. Extending the prediction horizon beyond next-day forecasts